In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from indoorplants.analysis import exploratory

%matplotlib inline

# 1.0 Exploratory Data Analysis


In [2]:
data_path = os.path.join(os.getcwd(),"home-credit-default-risk")

In [3]:
plt.style.use("seaborn-poster")

In [4]:
def clean_column_names(table):
    table.columns = table.columns.str.replace(r'[^\w\s]','') 
    table.columns = table.columns.str.strip().str.replace(' ','_').str.lower()
    return table

## 1.1 Application_Train

### 1.1.1 Shape

In [5]:
app_train = pd.read_csv(os.path.join(data_path,"application_train.csv"))

In [6]:
app_train = clean_column_names(app_train)

In [7]:
app_train.columns

Index(['sk_id_curr', 'target', 'name_contract_type', 'code_gender',
       'flag_own_car', 'flag_own_realty', 'cnt_children', 'amt_income_total',
       'amt_credit', 'amt_annuity',
       ...
       'flag_document_18', 'flag_document_19', 'flag_document_20',
       'flag_document_21', 'amt_req_credit_bureau_hour',
       'amt_req_credit_bureau_day', 'amt_req_credit_bureau_week',
       'amt_req_credit_bureau_mon', 'amt_req_credit_bureau_qrt',
       'amt_req_credit_bureau_year'],
      dtype='object', length=122)

In [8]:
app_train.shape

(307511, 122)

**Notes:**

There are 121 features in the application training data. 
`target` is the label we want to predict. 

- 1 -  the loan was not repaid
- 0 - the loan was repaid

In [9]:
app_train.target.value_counts()

0    282686
1     24825
Name: target, dtype: int64

In [10]:
app_train.target.value_counts(normalize = True)

0    0.919271
1    0.080729
Name: target, dtype: float64

We can tell that there are around 92% of the examples in the positive class (0) and there are around 8% of examples in the negative class (1).

### 1.1.2 Nulls

In [11]:
def get_null_stats(table):
    null_stats = table.isnull().sum().to_frame()
    null_stats = null_stats[(null_stats!=0).all(axis = 1)]
    null_stats.columns = ['num_of_null']
    null_stats['normalize_of_null'] = null_stats['num_of_null']/len(table)
    null_stats = null_stats.sort_values(by='num_of_null',ascending = False)
    return null_stats

In [12]:
null_stats = get_null_stats(table=app_train)
null_stats[:20]

,num_of_null,normalize_of_null
commonarea_medi,214865,0.698723
commonarea_avg,214865,0.698723
commonarea_mode,214865,0.698723
nonlivingapartments_medi,213514,0.694330
nonlivingapartments_mode,213514,0.694330
nonlivingapartments_avg,213514,0.694330
fondkapremont_mode,210295,0.683862
livingapartments_mode,210199,0.683550
livingapartments_medi,210199,0.683550
livingapartments_avg,210199,0.683550


**Notes:**

- I found that there are large amount of information about the clients are null (more than 50%). 
- This could be that the info of clients buildings are hard to get. 

- Therefore, building informations may not be ideal features to predict the default probability.

### 1.1.3 Duplicates

In [13]:
app_train.duplicated().sum()

0

There is NO duplicated examples in the training table.

### 1.1.4 Features

In [14]:
columns_list = app_train.columns.tolist()

In [15]:
columns_type = app_train.dtypes.to_frame()

In [16]:
columns_type.columns = ['data_type']

In [17]:
columns_type.data_type.value_counts()

float64    65
int64      41
object     16
Name: data_type, dtype: int64

Let's particularly take a look at the features with `object` type.

In [18]:
object_columns = columns_type[columns_type.data_type == 'object'].index.tolist()
object_columns

['name_contract_type',
 'code_gender',
 'flag_own_car',
 'flag_own_realty',
 'name_type_suite',
 'name_income_type',
 'name_education_type',
 'name_family_status',
 'name_housing_type',
 'occupation_type',
 'weekday_appr_process_start',
 'organization_type',
 'fondkapremont_mode',
 'housetype_mode',
 'wallsmaterial_mode',
 'emergencystate_mode']

In [19]:
for i in object_columns:
    print('\n',i,'\n')
    print(app_train[i].value_counts())


 name_contract_type 

Cash loans         278232
Revolving loans     29279
Name: name_contract_type, dtype: int64

 code_gender 

F      202448
M      105059
XNA         4
Name: code_gender, dtype: int64

 flag_own_car 

N    202924
Y    104587
Name: flag_own_car, dtype: int64

 flag_own_realty 

Y    213312
N     94199
Name: flag_own_realty, dtype: int64

 name_type_suite 

Unaccompanied      248526
Family              40149
Spouse, partner     11370
Children             3267
Other_B              1770
Other_A               866
Group of people       271
Name: name_type_suite, dtype: int64

 name_income_type 

Working                 158774
Commercial associate     71617
Pensioner                55362
State servant            21703
Unemployed                  22
Student                     18
Businessman                 10
Maternity leave              5
Name: name_income_type, dtype: int64

 name_education_type 

Secondary / secondary special    218391
Higher education                  

**Notes**
- Interesting information
- We will need to use one hot encoding for modelling later

#### 1.1.4.1 Number of Enquiries to Credit Bureau

In [20]:
bureau_list =[i for i in columns_list if 'bureau' in i]
bureau_list

['amt_req_credit_bureau_hour',
 'amt_req_credit_bureau_day',
 'amt_req_credit_bureau_week',
 'amt_req_credit_bureau_mon',
 'amt_req_credit_bureau_qrt',
 'amt_req_credit_bureau_year']

**Data Description**
- Number of enquiries to Credit Bureau about the client one hour before application
- Number of enquiries to Credit Bureau about the client one day before application (excluding one hour before application)
- Number of enquiries to Credit Bureau about the client one week before application (excluding one day before application)
- Number of enquiries to Credit Bureau about the client one month before application (excluding one week before application)
- Number of enquiries to Credit Bureau about the client 3 month before application (excluding one month before application)
- Number of enquiries to Credit Bureau about the client one day year (excluding last 3 months before application)

In [21]:
for i in bureau_list:
    print(app_train[i].value_counts())
    

0.0    264366
1.0      1560
2.0        56
3.0         9
4.0         1
Name: amt_req_credit_bureau_hour, dtype: int64
0.0    264503
1.0      1292
2.0       106
3.0        45
4.0        26
5.0         9
6.0         8
9.0         2
8.0         1
Name: amt_req_credit_bureau_day, dtype: int64
0.0    257456
1.0      8208
2.0       199
3.0        58
4.0        34
6.0        20
5.0        10
8.0         5
7.0         2
Name: amt_req_credit_bureau_week, dtype: int64
0.0     222233
1.0      33147
2.0       5386
3.0       1991
4.0       1076
5.0        602
6.0        343
7.0        298
9.0        206
8.0        185
10.0       132
11.0       119
12.0        77
13.0        72
14.0        40
15.0        35
16.0        23
17.0        14
18.0         6
19.0         3
24.0         1
27.0         1
22.0         1
23.0         1
Name: amt_req_credit_bureau_mon, dtype: int64
0.0      215417
1.0       33862
2.0       14412
3.0        1717
4.0         476
5.0          64
6.0          28
7.0           7
8.0 

#### 1.1.4.2 Clients' Building Information


**Notes**:

Normalized information about building where the client lives, 
What is average (_AVG suffix), 
modus (_MODE suffix), 
median (_MEDI suffix) 

- apartment size, 
- common area, 
- living area, 
- age of building, 
- number of elevators, 
- number of entrances, 
- state of the building, 
- number of floor

In [22]:
avg_list = [i for i in app_train.columns if 'avg' in i]
avg_list

['apartments_avg',
 'basementarea_avg',
 'years_beginexpluatation_avg',
 'years_build_avg',
 'commonarea_avg',
 'elevators_avg',
 'entrances_avg',
 'floorsmax_avg',
 'floorsmin_avg',
 'landarea_avg',
 'livingapartments_avg',
 'livingarea_avg',
 'nonlivingapartments_avg',
 'nonlivingarea_avg']

In [23]:
mode_list = [i for i in app_train.columns if 'mode' in i]

In [24]:
medi_list = [i for i in app_train.columns if 'medi' in i]

## 1.2 bureau.csv

In [25]:
bureau = pd.read_csv(os.path.join(data_path,"bureau.csv"))

In [26]:
bureau.shape

(1716428, 17)

In [27]:
bureau = clean_column_names(bureau)

In [28]:
bureau.columns

Index(['sk_id_curr', 'sk_id_bureau', 'credit_active', 'credit_currency',
       'days_credit', 'credit_day_overdue', 'days_credit_enddate',
       'days_enddate_fact', 'amt_credit_max_overdue', 'cnt_credit_prolong',
       'amt_credit_sum', 'amt_credit_sum_debt', 'amt_credit_sum_limit',
       'amt_credit_sum_overdue', 'credit_type', 'days_credit_update',
       'amt_annuity'],
      dtype='object')

In [29]:
null_stats = get_null_stats(table=bureau)

In [30]:
null_stats

,num_of_null,normalize_of_null
amt_annuity,1226791,0.714735
amt_credit_max_overdue,1124488,0.655133
days_enddate_fact,633653,0.369170
amt_credit_sum_limit,591780,0.344774
amt_credit_sum_debt,257669,0.150119
days_credit_enddate,105553,0.061496
amt_credit_sum,13,0.000008


In [31]:
bureau.duplicated().sum()

0

**Notes:**
- `amt_annuity`,`amt_credit_max_overdue`, `days_enddate_fact` and `amt_credit_sum_limit` have more than 30% of entries are null. 

- There is no duplicated rows in the table `bureau`

- `sk_id_bureau`: Recoded ID of previous Credit Bureau credit related to our loan (unique coding for each loan application)

In [35]:
# bureau.set_index('sk_id_bureau') 

## 1.3 bureau_balance.csv

In [36]:
bureau_balance = pd.read_csv(os.path.join(data_path,"bureau_balance.csv"))

In [37]:
bureau_balance.duplicated().sum()

0

In [38]:
null_stats = get_null_stats(table=bureau_balance)
null_stats

,num_of_null,normalize_of_null


**Notes:**
- There is no duplicates or nulls in the `bureau_balance` table


In [40]:
bureau_balance = clean_column_names(bureau_balance)

In [41]:
bureau_balance.columns

Index(['sk_id_bureau', 'months_balance', 'status'], dtype='object')

**Let's join `bureau_balance`  to `bureau` on `sk_id_bureau`**

In [45]:
bureau.head()

,sk_id_curr,sk_id_bureau,credit_active,credit_currency,days_credit,credit_day_overdue,days_credit_enddate,days_enddate_fact,amt_credit_max_overdue,cnt_credit_prolong,amt_credit_sum,amt_credit_sum_debt,amt_credit_sum_limit,amt_credit_sum_overdue,credit_type,days_credit_update,amt_annuity
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [47]:
joined = bureau.set_index('sk_id_bureau').join(bureau_balance.set_index(['sk_id_bureau']))

In [50]:
joined_null_stats = get_null_stats(table=joined)

In [51]:
joined_null_stats

,num_of_null,normalize_of_null
amt_credit_max_overdue,18130741,7.217130e-01
amt_credit_sum_limit,10671361,4.247846e-01
amt_annuity,10495530,4.177855e-01
days_enddate_fact,5978831,2.379936e-01
amt_credit_sum_debt,4234392,1.685544e-01
days_credit_enddate,1232569,4.906369e-02
months_balance,942074,3.750024e-02
status,942074,3.750024e-02
amt_credit_sum,13,5.174785e-07


In [53]:
joined.head()

,sk_id_curr,credit_active,credit_currency,days_credit,credit_day_overdue,days_credit_enddate,days_enddate_fact,amt_credit_max_overdue,cnt_credit_prolong,amt_credit_sum,amt_credit_sum_debt,amt_credit_sum_limit,amt_credit_sum_overdue,credit_type,days_credit_update,amt_annuity,months_balance,status
sk_id_bureau,,,,,,,,,,,,,,,,,,
5000000,166497,Closed,currency 1,-2918,0,-2613.0,-2639.0,NaN,0,29443.50,0.00,NaN,0.0,Consumer credit,-2512,NaN,NaN,NaN
5000001,166497,Closed,currency 1,-1015,0,-831.0,-891.0,2223.855,0,13810.50,0.00,0.0,0.0,Consumer credit,-891,NaN,NaN,NaN
5000002,166497,Closed,currency 1,-149,0,-26.0,-26.0,0.000,0,13455.00,0.00,0.0,0.0,Consumer credit,-23,NaN,NaN,NaN
5000003,166497,Closed,currency 1,-135,0,230.0,-42.0,0.000,0,37350.00,0.00,0.0,0.0,Consumer credit,-42,NaN,NaN,NaN
5000004,166497,Active,currency 1,-47,0,320.0,NaN,0.000,0,315127.62,315127.62,0.0,0.0,Consumer credit,-17,NaN,NaN,NaN
